In [ ]:
!pip install -q transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import RobertaTokenizer, RobertaModel
from sklearn.model_selection import train_test_split


In [ ]:
def OHE_split(split = False,reset_index = False):

    data = pd.read_csv("/content/drive/MyDrive/mtsamples.csv")
    data.drop(['Unnamed: 0'],axis=1,inplace=True)


    counts = data['medical_specialty'].value_counts()
    others = [k for k,v in counts.items() if v<100]
    # others

    data = data[data['medical_specialty'].isin(others)]


    counts = data['medical_specialty'].value_counts()
    # print(counts)

    num_classes = len(data['medical_specialty'].unique())
    class_dict = dict(zip(data['medical_specialty'].unique(),list(range(num_classes))))
    data['medical_specialty'] = data['medical_specialty'].apply(lambda x:class_dict[x])
    data.dropna(inplace=True)
    data['transcription'] = data['keywords']+data['transcription']
    X = data[['transcription']]
    y = data[["medical_specialty"]]

    df = pd.concat([X,y],axis=1)
    df = df.rename(columns={"medical_specialty":"list"})



    if split:
        train_val, test = train_test_split(df, test_size=0.1, random_state=42)
        train, val = train_test_split(train_val, test_size=0.1111, random_state=42)

        if reset_index:
            train = train.reset_index(drop=True)
            test = test.reset_index(drop=True)
            val = val.reset_index(drop=True)
            return train, test, val    # ***** returns tuple ***********

        else:
            return train, test, val    # ***** returns tuple ***********

    else:
        return df

In [ ]:
df = OHE_split(split=False)

In [ ]:
df

,transcription,list
0,"allergy / immunology, allergic rhinitis, aller...",0
1,"bariatrics, laparoscopic gastric bypass, weigh...",1
2,"bariatrics, laparoscopic gastric bypass, heart...",1
5,"bariatrics, gastric bypass, eea anastomosis, r...",1
6,"bariatrics, breast reconstruction, excess, lma...",1
...,...,...
4984,"bariatrics, laparoscopic gastric bypass, gastr...",1
4985,"bariatrics, jenny craig, medifast, nutrisystem...",1
4989,"bariatrics, elective surgical weight loss, sur...",1
4993,"allergy / immunology, chronic glossitis, xeros...",0


#### 65.103% of Data is less than 512 words

In [ ]:
sum(df['transcription'].str.split().str.len() < 512)/len(df) *100

71.39334155363748

In [ ]:
train,test,val= OHE_split(split=True,reset_index=True)
print(train.shape)
print(test.shape)
print(val.shape)

(648, 2)
(82, 2)
(81, 2)


In [ ]:
val

,transcription,list
0,"physical medicine - rehab, emg, nerve conducti...",8
1,"nephrology, end-stage renal disease, av fistul...",14
2,"ent - otolaryngology, bilateral myringotomies,...",20
3,"neurosurgery, thoracic right-sided discectomy,...",13
4,"ophthalmology, entropion, blepharon, catgut su...",11
...,...,...
76,"letters, cardiovascular, letter, angioedema, b...",15
77,"nephrology, nephrolithiasis, septic, lithotrip...",14
78,"hematology - oncology, chemo, taxol, intraperi...",19
79,"rheumatology, ra, rheumatoid arthritis, joint ...",5


In [ ]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
TEST_BATCH_SIZE = 4
EPOCHS = 4
LEARNING_RATE = 1e-05
tokenizer = RobertaTokenizer.from_pretrained('roberta-base',cache_dir='./')

In [ ]:
tokenizer("hi")

{'input_ids': [0, 3592, 2], 'attention_mask': [1, 1, 1]}

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.comment_text = self.data['transcription']
        # print(self.comment_text[1332])
        self.targets = self.data['list']
        self.max_len = max_len

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, index):
        comment_text = str(self.comment_text[index])
        comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer(
            comment_text,
            padding='max_length',
            max_length=self.max_len,
            truncation=True,
            return_token_type_ids=True

        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        # token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            # 'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': TEST_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }


val_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }


In [ ]:
def load_data(train,test,val):

    training_set = CustomDataset(train, tokenizer, MAX_LEN)
    testing_set = CustomDataset(test, tokenizer, MAX_LEN)
    val_set = CustomDataset(val, tokenizer, MAX_LEN)

    training_loader = DataLoader(training_set, **train_params)
    testing_loader = DataLoader(testing_set, **test_params)
    val_loader = DataLoader(val_set, **test_params)

    return training_loader,testing_loader,val_loader


In [ ]:
train_loader,test_loader,val_loader = load_data(train,test,val)

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
class RoBERTClass(torch.nn.Module):
    def __init__(self):
        super(RoBERTClass, self).__init__()
        self.l1 = transformers.RobertaModel.from_pretrained('roberta-base',cache_dir = './model/')
        # self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 27)

    def forward(self, ids, mask):
        _, output_1= self.l1(ids, attention_mask = mask, return_dict=False)
        # output_2 = self.l2(output_1)
        output = self.l3(output_1)
        return output

model = RoBERTClass()
model.to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RoBERTClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((7

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.CrossEntropyLoss()(outputs, targets)

In [ ]:
def train(epoch):
    model.train()
    for _,data in enumerate(train_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        # token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask)

        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if _%100==0:
            print(f'Epoch: {epoch},Batch: {_} Loss:  {loss.item()}')



In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

Epoch: 0,Batch: 0 Loss:  3.286590576171875
Epoch: 1,Batch: 0 Loss:  2.746624231338501
Epoch: 2,Batch: 0 Loss:  2.3070240020751953
Epoch: 3,Batch: 0 Loss:  0.6503286957740784


In [ ]:
def validation():
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(val_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            # token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
for epoch in range(EPOCHS):

    outputs, targets = validation()
    out_array = np.array(outputs)
    tar_array = np.array(targets)

    max_tar_indices = np.zeros_like(out_array)
    max_tar_indices[np.arange(tar_array.shape[0]),tar_array] = 1
    max_tar_indices = max_tar_indices.tolist()

    max_value_indices = np.zeros_like(out_array)
    max_indices = np.argmax(out_array, axis=1)
    max_value_indices[np.arange(out_array.shape[0]), max_indices] = 1
    max_value_indices = max_value_indices.tolist()


    acuracy = sum(max_indices == tar_array)/len(tar_array)
    print("accuracy",acuracy)

    # f1_score_None = metrics.f1_score(max_tar_indices,max_value_indices,average=None)
    f1_score = metrics.f1_score(max_tar_indices,max_value_indices,average="micro")

    print("f1_Score",f1_score)

accuracy 0.9753086419753086
f1_Score 0.9753086419753086
accuracy 0.9753086419753086
f1_Score 0.9753086419753086
accuracy 0.9753086419753086
f1_Score 0.9753086419753086
accuracy 0.9753086419753086
f1_Score 0.9753086419753086
